# Detect hand-to-mouth eating motion using accelerometer and gyroscope data

## Data Collection
Collect data using an accelerometer and gyroscope. Label the data as eating or not eating.

## Data Preprocessing
Preprocess data by normalizing and segmenting it into fixed-size windows. This helps the model learn temporal patterns.

## Feature Extraction
Extract statistical features.

## Model Training with LSTM

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# from google.colab import drive
# drive.mount('/content/drive')
# drive.mount("/content/drive", force_remount=True)

# %cd /content/drive/MyDrive

# Load your dataset
data = pd.read_csv('merged_sensor_data.csv')

# Assuming the dataset has columns: ['accel_z', 'accel_y', 'accel_x', 'gyro_z', 'gyro_y', 'gyro_x', 'label']
features = data[['accel_z', 'accel_y', 'accel_x', 'gyro_z', 'gyro_y', 'gyro_x']].values
labels = data['label'].values

# Encode the labels using LabelEncoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Normalize the data
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Split data into sequences
def create_sequences(data, labels, time_steps=50):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(labels[i+time_steps])
    return np.array(X), np.array(y)

time_steps = 100
X, y = create_sequences(features, labels, time_steps)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to numeric values
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(time_steps, X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Save the model
model.save('hand_to_mouth_model.h5')

2024-06-05 20:32:13.119717: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 20:32:14.010299: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/50


/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


58/58 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.7622 - loss: 0.5342 - val_accuracy: 0.8716 - val_loss: 0.2476
Epoch 2/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.8561 - loss: 0.2670 - val_accuracy: 0.9010 - val_loss: 0.2261
Epoch 3/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.8842 - loss: 0.2276 - val_accuracy: 0.9086 - val_loss: 0.2139
Epoch 4/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.8859 - loss: 0.2741 - val_accuracy: 0.9162 - val_loss: 0.1903
Epoch 5/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 61ms/step - accuracy: 0.9089 - loss: 0.1956 - val_accuracy: 0.7650 - val_loss: 0.4378
Epoch 6/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 0.7593 - loss: 0.4118 - val_accuracy: 0.7802 - val_loss: 0.3329
Epoch 7/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - accuracy: 0.7973 - loss: 0.3288 - val_accuracy: 0.8346 - val_loss: 0.3132
Epoch 8/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - accuracy: 0.7992 - loss: 0.3198 - val_accuracy: 0.8455 - val_loss: 0.

Test Accuracy: 93.73%


Trained Model has high Test Accuracy: 93.73%

## Deployment
Once the model is trained and saved, load it for predictions on new data:

In [2]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('hand_to_mouth_model.h5')

# Function to predict on new data segments
def predict_hand_to_mouth(segments):
    predictions = model.predict(segments)
    return predictions > 0.5  # you may adjust the threshold based on your evaluation

# Split data into sequences
def create_new_sequences(data, time_steps=50):
    X = []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
    return np.array(X)

# Load your dataset
new_data = pd.read_csv('validation_sensor_data.csv')

# Assuming the dataset has columns: ['accel_z', 'accel_y', 'accel_x', 'gyro_z', 'gyro_y', 'gyro_x']
new_features = new_data[['accel_z', 'accel_y', 'accel_x', 'gyro_z', 'gyro_y', 'gyro_x']].values

# Normalize the data
scaler = StandardScaler()
new_features = scaler.fit_transform(new_features)
time_steps = 100

# Preprocess new data
new_data= create_new_sequences(new_features, time_steps)

# new_segments = preprocess_new_data(new_df)

# Predict using the model
results = predict_hand_to_mouth(new_data)

# Post-process predictions
# Here, results is a list of boolean values indicating hand-to-mouth eating detection for each segment
for i, res in enumerate(results):
    if res:
        print(f"Segment {i}: Hand-to-mouth eating detected")
    else:
        print(f"Segment {i}: No hand-to-mouth eating detected")


94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step
Segment 0: Hand-to-mouth eating detected
Segment 1: Hand-to-mouth eating detected
Segment 2: Hand-to-mouth eating detected
Segment 3: Hand-to-mouth eating detected
Segment 4: Hand-to-mouth eating detected
Segment 5: Hand-to-mouth eating detected
Segment 6: Hand-to-mouth eating detected
Segment 7: Hand-to-mouth eating detected
Segment 8: Hand-to-mouth eating detected
Segment 9: Hand-to-mouth eating detected
Segment 10: Hand-to-mouth eating detected
Segment 11: Hand-to-mouth eating detected
Segment 12: Hand-to-mouth eating detected
Segment 13: Hand-to-mouth eating detected
Segment 14: Hand-to-mouth eating detected
Segment 15: Hand-to-mouth eating detected
Segment 16: Hand-to-mouth eating detected
Segment 17: Hand-to-mouth eating detected
Segment 18: Hand-to-mouth eating detected
Segment 19: Hand-to-mouth eating detected
Segment 20: Hand-to-mouth eating detected
Segment 21: Hand-to-mouth eating detected
Segment 22: Hand-to-mouth eating detected
Segm

## Summary
* Data Collection: Collect accelerometer and gyroscope data.
* Preprocessing: Normalize and segment data.
* Feature Extraction: Optional if using raw data with LSTM.
* Model Training: Use LSTM for training.
* Deployment: Use the trained model for real-time detection.